In [1]:
import locale
   ...: locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

import pandas as pd
import pyarrow
import plotly.express as px
import yaml
from IPython.display import display, Markdown
import datetime
import dateutil.relativedelta

#for key, value in locale.localeconv().items():
#    print("%s: %s" % (key, value))

## Acompanhamento das pessoas com ferida vascular

Essa seção se dedica a identificar como é o registro do acompanhamento de pessoas com ferida vascular no município.

As pessoas foram computadas se tivessem códigos CID-10, CIAP-2 ou ABP relacionados ao diagnóstico de feridas vasculares registrados, e avaliou-se se o acompanhamento e a troca de curativos está adequada.

In [2]:

display(Markdown("## Dados gerais de perfil do indicador"))

with open('P10_Ferida-vascular.yml', 'r') as file:
    indicador = yaml.safe_load(file)

#for P in indicador['perfil'].keys():
#    display(Markdown(f"**{P}:** {indicador['perfil'][P]}  "))
#    print(P + ": " + str(indicador['perfil'][P]) + " ")

display(Markdown(f"O indicador é do tipo **{indicador['perfil']['tipo']}**."))

display(Markdown(f"Serão avaliados registros de:"))

for P in indicador['perfil']['fichas']:
    display(Markdown(f"- {P}"))

display(Markdown(f"Realizados nos últimos {indicador['perfil']['meses']} meses."))


## Dados gerais de perfil do indicador

O indicador é do tipo **cobertura proporcional**.

Serão avaliados registros de:

- atendimento individual

- visita domiciliar

Realizados nos últimos 12 meses.

In [3]:


df = pd.read_parquet('Cidade_X/atendimentos.parquet')

display(Markdown("## Dados gerais da população atendida no perído de análise"))

# display(Markdown("### Todo período do banco"))

# Nota: preciso delimitar os últimos 12 meses apenas.

atendimentos_total = df.shape[0]
populacao_total = df['paciente_id'].nunique()

# Deixando o número em português

pt_atendimentos = locale.format_string('%u', atendimentos_total, grouping=True )
pt_populacao = locale.format_string('%u', populacao_total, grouping=True )

# display(Markdown(f"Total de atendimentos = **{pt_atendimentos}**.  "))
# display(Markdown(f"Pessoas atendidas = **{pt_populacao}**.  "))

agora = pd.Timestamp.now()
antiga = df.co_dim_tempo.min()
recente = df.co_dim_tempo.max()

display(Markdown("**Atualização do banco**  "))
display(Markdown(f"Data e horas de hoje: {agora.strftime('%Y-%m-%d')}  "))
display(Markdown(f"Registro mais antigo: {antiga}  "))
display(Markdown(f"Registro mais recente: {recente}  "))

display(Markdown(f"**Período para apuração do indicador:**  {indicador['perfil']['meses']} meses"))


inicio = agora - dateutil.relativedelta.relativedelta(months=int(indicador['perfil']['meses']))



## Dados gerais da população atendida no perído de análise

**Atualização do banco**  

Data e horas de hoje: 2024-01-30  

Registro mais antigo: 2011-01-01  

Registro mais recente: 2024-01-21  

**Período para apuração do indicador:**  12 meses

In [4]:
display(Markdown("### Período de apuração do indicador"))

display(Markdown(f"Serão considerados os atendimentos a partir de **{inicio.strftime('%Y-%m-%d')}**"))

df['co_dim_tempo'] = pd.to_datetime(df['co_dim_tempo'])
df = df.loc[(df['co_dim_tempo'] >= inicio)]

atendimentos_total = df.shape[0]
populacao_total = df['paciente_id'].nunique()

# Deixando o número em português

pt_atendimentos = locale.format_string('%u', atendimentos_total, grouping=True )
pt_populacao = locale.format_string('%u', populacao_total, grouping=True )

display(Markdown(f"Total de atendimentos em {indicador['perfil']['meses']} meses = **{pt_atendimentos}**.  "))
display(Markdown(f"Pessoas atendidas {indicador['perfil']['meses']} meses= **{pt_populacao}**.  "))


### Período de apuração do indicador

Serão considerados os atendimentos a partir de **2023-01-30**

Total de atendimentos em 12 meses = **134.773**.  

Pessoas atendidas 12 meses= **37.358**.  

In [5]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df):
    # Performed 1 aggregation grouped on column: 'paciente_id'
    df = df.groupby(['paciente_id']).agg(ds_filtro_proced_avaliados_count=('ds_filtro_proced_avaliados', 'count')).reset_index()
    return df

# df_clean = clean_data(df.copy())
# df_clean.head()

In [6]:
# df.dtypes

In [7]:


display(Markdown("## Cálculo do denominador"))

display(Markdown("### Atendimentos codificados com CID-10"))

df1 = pd.DataFrame()
df2 = pd.DataFrame()

for C in indicador['denominador']['filtro_cids']:
    df1 = df[df['ds_filtro_cids'].str.contains(C)]
    N_atend = df1.shape[0]
    display(Markdown(f"CID {C} encontrados **{N_atend}**  "))
    df2 = pd.concat([df1, df2], ignore_index=True).drop_duplicates()

atd_cid = df2.shape[0]
pessoas_cid = df2['paciente_id'].nunique()

display(Markdown(f"Atendimentos por {indicador['perfil']['nome_indicador']}, removendo duplicados = **{atd_cid}**.  "))
display(Markdown(f"Pessoas com {indicador['perfil']['nome_indicador']} encontradas via CID-10 = **{pessoas_cid}**.  "))

## Cálculo do denominador

### Atendimentos codificados com CID-10

CID L97 encontrados **60**  

CID L98.4 encontrados **0**  

Atendimentos por Ferida vascular, removendo duplicados = **60**.  

Pessoas com Ferida vascular encontradas via CID-10 = **40**.  

In [8]:


display(Markdown("### Atendimentos codificados com CIAP-2 ou ABP"))

df1 = pd.DataFrame()
df3 = pd.DataFrame()


for C in indicador['denominador']['filtro_ciaps']:
    df1 = df[df['ds_filtro_ciaps'].str.contains(C)]
    N_atend = df1.shape[0]
    display(Markdown(f" {C} encontrados **{N_atend}**  "))
    df3 = pd.concat([df1, df3], ignore_index=True).drop_duplicates()

atd_ciap = df3.shape[0]
pessoas_ciap = df3['paciente_id'].nunique()

display(Markdown(f"Atendimentos por {indicador['perfil']['nome_indicador']}, removendo duplicados = **{atd_ciap}**."))
display(Markdown(f"Pessoas com {indicador['perfil']['nome_indicador']} encontradas via CID-10 = **{pessoas_ciap}**."))

### Atendimentos codificados com CIAP-2 ou ABP

 S97 encontrados **266**  

Atendimentos por Ferida vascular, removendo duplicados = **266**.

Pessoas com Ferida vascular encontradas via CID-10 = **128**.

In [9]:


display(Markdown(f"### Total de pessoas para: {indicador['perfil']['nome_indicador']}"))

lista_denominador = pd.concat([df2, df3], ignore_index=True).drop_duplicates()

atd_alvo = lista_denominador.shape[0]
populacao_alvo = lista_denominador['paciente_id'].nunique()

display(Markdown(f"Total de atendimentos pela condição alvo = **{atd_alvo}**.  "))
display(Markdown(f"Total de pessoas na população alvo = **{populacao_alvo}**."))

### Total de pessoas para: Ferida vascular

Total de atendimentos pela condição alvo = **319**.  

Total de pessoas na população alvo = **150**.

In [10]:


import plotly.graph_objects as go

labels = ['População alvo','População total']

values = [populacao_alvo, populacao_total]

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_traces(marker=dict(colors=['78b4d1', '2775b1']))
fig.show()

percentual = populacao_alvo / populacao_total * 100

percentual = locale.format_string('%10.2f', percentual)

display(Markdown(f"Sua população-alvo para esse indicador corresponde a {percentual}% da população total."))


Sua população-alvo para esse indicador corresponde a       0,40% da população total.

## Cálculo do Numerador

In [11]:


display(Markdown(f"### Atendimentos com abordagem individual de {indicador['perfil']['nome_indicador']}"))

df0 = lista_denominador
df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()

display(Markdown("### Perfil da Condição A"))

for P in indicador['condição_A'].keys():
    display(Markdown(f"{P}: {indicador['condição_A'][P]}  "))


### Atendimentos com abordagem individual de Ferida vascular

### Perfil da Condição A

descrição: ao menos 1 atendimento com aferição de peso e altura  

fonte: atendimento individual  

profissional: ['medico', 'enfermeiro']  

nu_peso: NOT_NULL  

nu_altura: NOT_NULL  

valor_min: 1  

In [13]:
display(Markdown("### Avaliação da Condição A na população-alvo:"))

display(Markdown("**Peso:**  "))

df1 = df0[df0.nu_peso > 0]

n_atd_peso = df1.shape[0]
pessoas_beneficiadas = df1['paciente_id'].nunique()

display(Markdown(f"Atendimentos com medição de peso: **{n_atd_peso}**.  "))
display(Markdown(f"Pessoas que foram pesadas = **{pessoas_beneficiadas}**."))

# Pensar na fórmula para quando o N necessário for maior do que 1. 


### Avaliação da Condição A na população-alvo:

**Peso:**  

Atendimentos com medição de peso: **3**.  

Pessoas que foram pesadas = **3**.

In [14]:
display(Markdown("**Altura:**  "))

df2 = df0[df0.nu_altura > 0]

n_atd_altura = df2.shape[0]
pessoas_beneficiadas1 = df2['paciente_id'].nunique()

display(Markdown(f"Atendimentos com medição de altura: **{n_atd_altura}**.  "))
display(Markdown(f"Pessoas que foram medidas = **{pessoas_beneficiadas1}**."))

# Pensar na fórmula para quando o N necessário for maior do que 1. 


**Altura:**  

Atendimentos com medição de altura: **3**.  

Pessoas que foram medidas = **3**.

In [15]:
display(Markdown("**Peso e altura:**  "))

df3 = df1[df1.nu_altura > 0]
df_condição_A = df3

n_atd = df3.shape[0]

pessoas_beneficiadas_A = df3['paciente_id'].nunique()

display(Markdown(f"Total de atendimentos com medição de peso e altura: **{n_atd}**.  "))
display(Markdown(f"Pessoas que foram medidas = **{pessoas_beneficiadas_A}**."))

# Pensar na fórmula para quando o N necessário for maior do que 1. 


**Peso e altura:**  

Total de atendimentos com medição de peso e altura: **3**.  

Pessoas que foram medidas = **3**.

In [16]:

labels = ['Condição A','População-alvo']
values = [pessoas_beneficiadas_A, populacao_alvo-pessoas_beneficiadas_A]

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_traces(marker=dict(colors=['78b4d1', '2775b1']))
fig.show()

percentual = pessoas_beneficiadas_A / populacao_alvo * 100

percentual = locale.format_string('%10.2f', percentual)

display(Markdown(f"{percentual}% das pessoas com {indicador['perfil']['nome_indicador']} foram pesadas e medidas."))


      2,00% das pessoas com Ferida vascular foram pesadas e medidas.

In [17]:

df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()

display(Markdown("### Perfil da Condição B"))

for P in indicador['condição_B'].keys():
    display(Markdown(f"{P}: {indicador['condição_B'][P]}  "))


### Perfil da Condição B

descrição: no mínimo 4 atendimentos anuais com enfermeiro para curativo  

fonte: atendimento individual  

profissional: enfermeiro  

procedimentos_solicitados: None  

procedimentos_avaliados: ['03.01.10.027-6', '03.01.10.028-4', '04.01.01.001-5']  

valor_min: 4  

In [19]:
display(Markdown("### Avaliação da Condição B:"))

df3 = pd.DataFrame()

for C in indicador['condição_B']['procedimentos_avaliados']:
    C = C.replace(".", "")
    C = C.replace("-", "")
    df1 = df[df['ds_filtro_proced_avaliados'].str.contains(C)]
    N_atend = df1.shape[0]
    display(Markdown(f"Procedimentos {C} avaliados **{N_atend}**  "))
    df3 = pd.concat([df1, df3], ignore_index=True).drop_duplicates()

procedimentos = df3.shape[0]
pessoas_beneficiadas = df3['paciente_id'].nunique()

df_pacientes_procedimentos = df3.groupby(['paciente_id']).agg(ds_filtro_proced_avaliados_count=('ds_filtro_proced_avaliados', 'count')).reset_index()
df_condição_B = df_pacientes_procedimentos[df_pacientes_procedimentos.ds_filtro_proced_avaliados_count >= int(indicador['condição_B']['valor_min'])]
pessoas_beneficiadas_B = df_condição_B.shape[0]

display(Markdown(f"Total de atendimentos com avaliação do curativo da ferida vascular = **{procedimentos}**.  "))
display(Markdown(f"Pessoas beneficiadas com ao menos 1 curativo = **{pessoas_beneficiadas}**."))
display(Markdown(f"Pessoas beneficiadas com ao menos 4 curativos = **{pessoas_beneficiadas_B}**."))

# Pensar na fórmula para quando o N necessário for maior do que 1. 


### Avaliação da Condição B:

Procedimentos 0301100276 avaliados **0**  

Procedimentos 0301100284 avaliados **0**  

Procedimentos 0401010015 avaliados **0**  

Total de atendimentos com avaliação do curativo da ferida vascular = **0**.  

Pessoas beneficiadas com ao menos 1 curativo = **0**.

Pessoas beneficiadas com ao menos 4 curativos = **0**.

In [20]:

display(Markdown("### Cálculo do indicador"))

display(Markdown(f"População-alvo: {populacao_alvo}"))

df_indicador = pd.merge(df_condição_A, df_condição_B, on='paciente_id')
                        
pessoas_beneficiadas = df_indicador.shape[0]

Pessoas_não_atendidas = populacao_alvo - pessoas_beneficiadas

labels = ['Pessoas não atendidas', 'Pessoas beneficiadas']
values = [Pessoas_não_atendidas, pessoas_beneficiadas]
colors = ['blue', 'white']

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_traces(marker=dict(colors=['2775b1', '78b4d1']))
fig.show()

indicador = pessoas_beneficiadas / populacao_alvo * 100

indicador = locale.format_string('%10.2f', indicador)

if pessoas_beneficiadas == 0:
    display(Markdown("Nenhuma pessoa recebeu o tratamento preconizado neste indicador"))
else:
    display(Markdown(f"{indicador}% da população analisada recebeu o tratamento preconizado neste indicador."))

# Pensar na fórmula para quando o N necessário for maior do que 1. 


### Cálculo do indicador

População-alvo: 150

Nenhuma pessoa recebeu o tratamento preconizado neste indicador

In [21]:
# df_indicador